In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Get initial list: https://www.wikiwand.com/en/List_of_current_United_States_Senators
# For ease of manipulation, plug into Google Sheets and then export to .csv
# Remove unnecessary columns
# Add in needed columns and clean data
# BA University, BA University Public/Private, BA University White %, BA University Lat/Lng, DeVos Vote

def clean_INSTNM(school):
    ba = [school for school in school.split("\n") if "(B" in school]
    if len(ba) == 0: # John Boozeman
        ba = [school for school in school.split("\n") if "(O" in school]
    if len(ba) == 0: # Rand Paul
        ba = [school for school in school.split("\n") if "(M" in school]
    ba = ba[0].split("(")[0]
    if ba[-1] == " ":
        ba = ba[:-1]
    return ba

def clean_party(party):
    if party == "Independent[1]": # Bernie Sanders
        return "Independent"
    else:
        return party

# https://www.nytimes.com/interactive/2017/02/07/us/politics/betsy-devos-confirmation-vote.html
def devos(row):
    senator = row["Name"]
    party = row["Party"]
    if senator == "Lisa Murkowski" or senator == "Susan Collins":
        return "No"
    if party == "Democratic" or party == "Independent":
        return "No"
    else:
        return "Yes"
    
senators = pd.read_csv("senators.csv")
senators = senators[["State", "Name", "Party", "Education"]]
# CHANGE TO FUNCTION THAT ACCOUNTS FOR 'ATTENDED'
senators["INSTNM"] = senators["Education"].apply(lambda x: clean_INSTNM(x))
senators["Party"] = senators["Party"].apply(lambda x: clean_party(x))
senators["DeVos"] = senators.apply(lambda x: devos(x), axis=1)

senators.head()

,State,Name,Party,Education,INSTNM,DeVos
0,Alabama,Richard Shelby,Republican,"University of Alabama, Tuscaloosa(BA; LLB)\nBi...","University of Alabama, Tuscaloosa",Yes
1,Alabama,Luther Strange,Republican,"Tulane University(BA, JD)",Tulane University,Yes
2,Alaska,Lisa Murkowski,Republican,Georgetown University(BA)\nWillamette Universi...,Georgetown University,No
3,Alaska,Dan Sullivan,Republican,Harvard University(BA)\nGeorgetown University(...,Harvard University,Yes
4,Arizona,John McCain,Republican,United States Naval Academy (BS),United States Naval Academy,Yes


In [3]:
# University Data
# https://catalog.data.gov/dataset/college-scorecard

# "CONTROL": 1 = Public, 2 = Private nonprofit, 3 = Private for-profit
def clean_CONTROL(number):
    if number == 1:
        return "Public"
    elif number == 2:
        return "Private nonprofit"
    elif number == 3:
        return "Private for-profit"

universities = pd.read_csv("CollegeScorecard_Raw_Data/MERGED2014_15_PP.csv")
universities = universities[["INSTNM", "CITY", "STABBR", "ZIP", "LATITUDE", "LONGITUDE", "CONTROL", 
                             "UGDS", "UGDS_WHITE", "COSTT4_A"]]
universities["CONTROL"] = universities["CONTROL"].apply(lambda x: clean_CONTROL(x))
# "UG": Number of undergrads

universities.head()

/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6,9,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1411,1425,1427,1503,1517,1529,1530,1532,1537,1540,1541,1542,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1605,1606,1609,1610,1613,1614,1615,1725,1726,1727,1728,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,INSTNM,CITY,STABBR,ZIP,LATITUDE,LONGITUDE,CONTROL,UGDS,UGDS_WHITE,COSTT4_A
0,Alabama A & M University,Normal,AL,35762,34.783368,-86.568502,Public,4206,0.0333,21475
1,University of Alabama at Birmingham,Birmingham,AL,35294-0110,33.502230,-86.809170,Public,11383,0.5922,20621
2,Amridge University,Montgomery,AL,36117-3553,32.362609,-86.174010,Private nonprofit,291,0.2990,16370
3,University of Alabama in Huntsville,Huntsville,AL,35899,34.722818,-86.638420,Public,5451,0.6988,21107
4,Alabama State University,Montgomery,AL,36104-0271,32.364317,-86.295677,Public,4811,0.0158,18184


In [19]:
# Merge 'senators' with 'universities' on college name
# Because of formatting issues, let's make sure the names line up

def correct_instnm(previous):
    if previous == "Tulane University":
        return "Tulane University of Louisiana"
    if previous == "University of Alabama, Tuscaloosa":
        return "The University of Alabama"
    if previous == "University of Arkansas, Fayetteville":
        return "University of Arkansas"
    if previous == "Colorado State University, Fort Collins":
        return "Colorado State University-Fort Collins"
    if previous == "University of Wisconsin, Milwaukee":
        return "University of Wisconsin-Milwaukee"
    if previous == "Colorado State University, Fort Collins":
        return "Colorado State University-Fort Collins"
    if previous == "University of Hawaii, Manoa":
        return "University of Hawaii at Manoa"
    if previous == "University of Mississippi, Oxford":
        return "University of Mississippi"
    if previous == "Louisiana State University, Baton Rogue":
        return "Louisiana State University and Agricultural & Mechanical College"
    if previous == "Kansas State University, Manhattan":
        return "Kansas State University"
    if previous == "Georgia Institute of Technology":
        return "Georgia Institute of Technology-Main Campus"
    if previous == "Washington State University, Pullman":
        return "Washington State University"
    if previous == "West Virginia University, Morgantown":
        return "West Virginia University"
    if previous == "University of Minnesota, Twin Cities":
        return "University of Minnesota-Twin Cities"
    if previous == "University of Missouri, Columbia":
        return "University of Missouri-Columbia"
    if previous == "University of Tennessee, Knoxville":
        return "The University of Tennessee-Knoxville"
    if previous == "University of South Carolina, Columbia":
        return "University of South Carolina-Columbia"
    if previous == "University of California, Santa Barbara":
        return "University of California-Santa Barbara"
    if previous == "University of Texas, Austin":
        return "The University of Texas at Austin"
    if previous == "University of Nevada, Reno":
        return "University of Nevada-Reno"
    if previous == "University of Nebraska, Lincoln":
        return "University of Nebraska-Lincoln"
    if previous == "Montana State University, Bozeman":
        return "Montana State University"
    if previous == "Ohio State University":
        return "Ohio State University-Main Campus"
    if previous == "Trinity University, Texas":
        return "Trinity University"
    if previous == "Shippensburg University":
        return "Shippensburg University of Pennsylvania"
    if previous == "University of Pittsburgh":
        return "University of Pittsburgh-Pittsburgh Campus"
    if previous == "St. Lawrence University":
        return "St Lawrence University"
    if previous == "Brigham Young University, Utah":
        return "Brigham Young University-Provo"
    if previous == "Southwest Baptist University, Bolivar":
        return "Southwest Baptist University"
    if previous == "St. Michael's College":
        return "Saint Michael's College"
    if previous == "University of Idaho, Moscow":
        return "University of Idaho"
    if previous == "St. Peter's University":
        return "Saint Peter's University"
    if previous == "University of Kansas, Lawrence":
        return "University of Kansas"
    if previous == "University of Maryland, University College":
        return "University of Maryland-University College"
    if previous == "Miami University":
        return "Miami University-Oxford"
    else:
        return previous

senators["INSTNM"] = senators["INSTNM"].apply(lambda x: correct_instnm(x))
senator_univs = pd.merge(senators, universities, on="INSTNM", how="left")

# https://www.wikiwand.com/en/United_States_Naval_Academy
senator_univs.loc[senator_univs['INSTNM'] == "United States Naval Academy", "CITY"] = "Annapolis"
senator_univs.loc[senator_univs['INSTNM'] == "United States Naval Academy", "STABBR"] = "MD"
senator_univs.loc[senator_univs['INSTNM'] == "United States Naval Academy", "CONTROL"] = "Military"
senator_univs.loc[senator_univs['INSTNM'] == "United States Naval Academy", "LATITUDE"] = "38.9820658"
senator_univs.loc[senator_univs['INSTNM'] == "United States Naval Academy", "LONGITUDE"] = "-76.4839405"
senator_univs.loc[senator_univs['INSTNM'] == "United States Naval Academy", "UGDS"] = "4576"

# https://www.wikiwand.com/en/United_States_Military_Academy
senator_univs.loc[senator_univs['INSTNM'] == "United States Military Academy", "CITY"] = "West Point"
senator_univs.loc[senator_univs['INSTNM'] == "United States Military Academy", "STABBR"] = "NY"
senator_univs.loc[senator_univs['INSTNM'] == "United States Military Academy", "CONTROL"] = "Military"
senator_univs.loc[senator_univs['INSTNM'] == "United States Military Academy", "LATITUDE"] = "41.3918372"
senator_univs.loc[senator_univs['INSTNM'] == "United States Military Academy", "LONGITUDE"] = "-73.9625033"
senator_univs.loc[senator_univs['INSTNM'] == "United States Military Academy", "UGDS"] = "4348"


,State,Name,Party,Education,INSTNM,DeVos,CITY,STABBR,ZIP,LATITUDE,LONGITUDE,CONTROL,UGDS,UGDS_WHITE,COSTT4_A
0,Alabama,Richard Shelby,Republican,"University of Alabama, Tuscaloosa(BA; LLB)\nBi...",The University of Alabama,Yes,Tuscaloosa,AL,35487-0166,33.2144,-87.5458,Public,29851,0.7825,26400
1,Alabama,Luther Strange,Republican,"Tulane University(BA, JD)",Tulane University of Louisiana,Yes,New Orleans,LA,70118-5698,29.9397,-90.1204,Private nonprofit,7892,0.7138,61055
2,Alaska,Lisa Murkowski,Republican,Georgetown University(BA)\nWillamette Universi...,Georgetown University,No,Washington,DC,20057-0001,38.9088,-77.0735,Private nonprofit,7211,0.5813,62179
3,Alaska,Dan Sullivan,Republican,Harvard University(BA)\nGeorgetown University(...,Harvard University,Yes,Cambridge,MA,02138,42.3744,-71.1182,Private nonprofit,7236,0.4527,59950
4,Arizona,John McCain,Republican,United States Naval Academy (BS),United States Naval Academy,Yes,Annapolis,MD,NaN,38.9820658,-76.4839405,Military,4576,NaN,NaN
5,Arizona,Jeff Flake,Republican,"Brigham Young University, Utah (BA; MA)",Brigham Young University-Provo,Yes,Provo,UT,84602,40.2509,-111.649,Private nonprofit,27163,0.8320,17236
6,Arkansas,John Boozman,Republican,"University of Arkansas, Fayetteville(attended)...",Southern College of Optometry,Yes,Memphis,TN,38104-2211,35.1383,-90.0199,Private nonprofit,NaN,NaN,NaN
7,Arkansas,Tom Cotton,Republican,Harvard University(BA; JD)\nClaremont Graduate...,Harvard University,Yes,Cambridge,MA,02138,42.3744,-71.1182,Private nonprofit,7236,0.4527,59950
8,California,Dianne Feinstein,Democratic,Stanford University(BA),Stanford University,No,Stanford,CA,94305,37.427,-122.169,Private nonprofit,7018,0.3752,60311
9,California,Kamala Harris,Democratic,Howard University(BA)\nUniversity of Californi...,Howard University,No,Washington,DC,20059-0001,38.9207,-77.0195,Private nonprofit,6961,0.0144,41643


In [39]:
def count_INSTNM(school):
    schools = [school for school in school.split("\n")]
    templist = []
    for attended in schools:
        attended = attended[attended.find("(")+1:attended.find(")")]
#         print attended
        if ";" in attended:
            templist.extend(attended.split(";"))
        elif "," in attended:
            templist.extend(attended.split(","))
        else:
            templist.append(attended)
    templist = filter(lambda a: a != "attended", templist)
    return len(templist)


senator_univs["countINSTNM"] = senator_univs["Education"].apply(lambda x: count_INSTNM(x))
senator_univs.head()

,State,Name,Party,Education,INSTNM,DeVos,CITY,STABBR,ZIP,LATITUDE,LONGITUDE,CONTROL,UGDS,UGDS_WHITE,COSTT4_A,countINSTNM
0,Alabama,Richard Shelby,Republican,"University of Alabama, Tuscaloosa(BA; LLB)\nBi...",The University of Alabama,Yes,Tuscaloosa,AL,35487-0166,33.2144,-87.5458,Public,29851,0.7825,26400,3
1,Alabama,Luther Strange,Republican,"Tulane University(BA, JD)",Tulane University of Louisiana,Yes,New Orleans,LA,70118-5698,29.9397,-90.1204,Private nonprofit,7892,0.7138,61055,2
2,Alaska,Lisa Murkowski,Republican,Georgetown University(BA)\nWillamette Universi...,Georgetown University,No,Washington,DC,20057-0001,38.9088,-77.0735,Private nonprofit,7211,0.5813,62179,2
3,Alaska,Dan Sullivan,Republican,Harvard University(BA)\nGeorgetown University(...,Harvard University,Yes,Cambridge,MA,02138,42.3744,-71.1182,Private nonprofit,7236,0.4527,59950,3
4,Arizona,John McCain,Republican,United States Naval Academy (BS),United States Naval Academy,Yes,Annapolis,MD,NaN,38.9820658,-76.4839405,Military,4576,NaN,NaN,1


In [64]:
# https://www.kaggle.com/mylesoneill/world-university-rankings
# Times Higher Education Ranking
# rankings = pd.read_csv('timesData.csv')

# Manually scraped from US NEWS
rankings = pd.read_csv('usnews_ranks.csv')
rankings = rankings[["INSTNM", "rankUSNEWSScore"]].set_index('INSTNM')['rankUSNEWSScore'].to_dict()

def get_usnews_INSTNM_rank(school):
    return rankings[school] 

senator_univs["rankUSNewsScore"] = senator_univs["INSTNM"].apply(lambda x: get_usnews_INSTNM_rank(x))
senator_univs.head()[["Name", "Party", "INSTNM", "rankUSNewsScore"]]

,Name,Party,INSTNM,rankUSNewsScore
0,Richard Shelby,Republican,The University of Alabama,47
1,Luther Strange,Republican,Tulane University of Louisiana,64
2,Lisa Murkowski,Republican,Georgetown University,78
3,Dan Sullivan,Republican,Harvard University,98
4,John McCain,Republican,United States Naval Academy,86


In [65]:
senator_univs.to_csv("senators_univs.csv")
senator_univs

,State,Name,Party,Education,INSTNM,DeVos,CITY,STABBR,ZIP,LATITUDE,LONGITUDE,CONTROL,UGDS,UGDS_WHITE,COSTT4_A,countINSTNM,rankScore,rankTimesScore,rankUSNewsScore
0,Alabama,Richard Shelby,Republican,"University of Alabama, Tuscaloosa(BA; LLB)\nBi...",The University of Alabama,Yes,Tuscaloosa,AL,35487-0166,33.2144,-87.5458,Public,29851,0.7825,26400,3,None,47,47
1,Alabama,Luther Strange,Republican,"Tulane University(BA, JD)",Tulane University of Louisiana,Yes,New Orleans,LA,70118-5698,29.9397,-90.1204,Private nonprofit,7892,0.7138,61055,2,None,64,64
2,Alaska,Lisa Murkowski,Republican,Georgetown University(BA)\nWillamette Universi...,Georgetown University,No,Washington,DC,20057-0001,38.9088,-77.0735,Private nonprofit,7211,0.5813,62179,2,58.8,78,78
3,Alaska,Dan Sullivan,Republican,Harvard University(BA)\nGeorgetown University(...,Harvard University,Yes,Cambridge,MA,02138,42.3744,-71.1182,Private nonprofit,7236,0.4527,59950,3,91.6,98,98
4,Arizona,John McCain,Republican,United States Naval Academy (BS),United States Naval Academy,Yes,Annapolis,MD,NaN,38.9820658,-76.4839405,Military,4576,NaN,NaN,1,None,86,86
5,Arizona,Jeff Flake,Republican,"Brigham Young University, Utah (BA; MA)",Brigham Young University-Provo,Yes,Provo,UT,84602,40.2509,-111.649,Private nonprofit,27163,0.8320,17236,2,None,57,57
6,Arkansas,John Boozman,Republican,"University of Arkansas, Fayetteville(attended)...",Southern College of Optometry,Yes,Memphis,TN,38104-2211,35.1383,-90.0199,Private nonprofit,NaN,NaN,NaN,1,None,0,0
7,Arkansas,Tom Cotton,Republican,Harvard University(BA; JD)\nClaremont Graduate...,Harvard University,Yes,Cambridge,MA,02138,42.3744,-71.1182,Private nonprofit,7236,0.4527,59950,2,91.6,98,98
8,California,Dianne Feinstein,Democratic,Stanford University(BA),Stanford University,No,Stanford,CA,94305,37.427,-122.169,Private nonprofit,7018,0.3752,60311,1,93.9,95,95
9,California,Kamala Harris,Democratic,Howard University(BA)\nUniversity of Californi...,Howard University,No,Washington,DC,20059-0001,38.9207,-77.0195,Private nonprofit,6961,0.0144,41643,2,-,43,43


In [5]:
# Now to our analyses! 
senator_univs["CONTROL"] = senator_univs["CONTROL"].fillna("Military")
senator_univs.groupby(["DeVos", "CONTROL"]).count()[["Name"]]

# Republicans went to more public schools, as a percentage, than Democrats

Name
DeVos CONTROL                
No    Military              1
      Private nonprofit    34
      Public               15
Yes   Military              2
      Private nonprofit    26
      Public               22

In [36]:
senator_univs.groupby(["DeVos", "Party"]).median()

# Republicans also went to whiter schools, but paid less
# Lot of land-grant public universities, maybe?

UGDS_WHITE      COSTT4_A  countINSTNM
DeVos Party                                             
No    Democratic     0.548142  45848.600000     2.043478
      Independent    0.464750  64133.500000     1.500000
      Republican     0.686550  60864.500000     1.500000
Yes   Republican     0.653591  36886.765957     1.860000

In [63]:
# Democrats are slightly more educated

senator_univs.groupby("Party").mean()

,UGDS_WHITE,COSTT4_A,countINSTNM,rankTimesScore
Party,,,,
Democratic,0.548142,45848.60000,2.043478,68.217391
Independent,0.464750,64133.50000,1.500000,93.500000
Republican,0.654937,37865.44898,1.846154,59.961538


In [7]:
# The story is that, ...